In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
__file__ = os.getcwd()
path_data_train = os.path.join(os.path.realpath(__file__).rsplit('/', 1)[0], 'preprocess', 'train_preprocess_2.csv')
path_label = os.path.join(os.path.realpath(__file__).rsplit('/', 1)[0], 'raw', 'train_label.csv')

In [3]:
X_train = pd.read_csv(path_data_train, engine = 'python')
y_train = pd.read_csv(path_label, engine = 'python')

In [4]:
# 잔존 고객 0, 이탈 고객 1
y_train['leave'] = 0
y_train['leave'][y_train.survival_time < 64] = 1

/home/user3/anaconda3/bigcontest/model:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
# 결제 기록이 없으면 0, 결제 기록이 있으면 1
y_train['amount'] = 0
y_train['amount'][y_train.amount_spent > 0] = 1

/home/user3/anaconda3/bigcontest/model:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.25, random_state = 42)
for train_index, test_index in split.split(y_train, y_train['leave']):
    strat_y_train = y_train.loc[train_index]
    strat_y_test = y_train.loc[test_index]

In [7]:
y_train = strat_y_train.sort_values(['acc_id'], ascending = True)
y_test = strat_y_test.sort_values(['acc_id'], ascending = True)

# 모델링

모델링 과정에서 교차검증을 통해 얻은 hyperparameter를 그대로 사용하지 않고 근처의 값을 사용하였는데, 이는 실제 평가 척도인 기대 이익에 최적화하는 과정에서 더 나은 결과를 보이는 hyperparameter를 사용하였기 때문입니다.

## 이탈 여부

### RandomForest

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier(random_state=42)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [14, 15, 16, 17]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)

grid_search.fit(X_train.iloc[:, 1:], y_train['leave'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:  2.1min remaining:  2.4min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.8min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'m

In [9]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7696333333333333
{'max_features': 16, 'n_estimators': 750}


In [10]:
from sklearn.externals import joblib

rf_leave_clf = RandomForestClassifier(random_state = 42, max_features = 15, n_estimators = 750, n_jobs = -1)
rf_leave_clf.fit(X_train.iloc[:, 1:], y_train['leave'])
joblib.dump(rf_leave_clf, './rf_leave_clf.pkl')

/home/user3/anaconda3/envs/hun/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['./rf_leave_clf.pkl']

### XGBoost

In [11]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

xgb = XGBClassifier(random_state=42)

param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'max_depth' : [13, 14, 15, 16], 'gamma' : [0, 1, 3]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)

grid_search.fit(X_train.iloc[:, 1:], y_train['leave'])

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  6.9min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=42, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'gamma': [0, 1, 3],
                          'learning_rate': [0.001, 0.01, 0.1],
                          'max_depth': [13, 14, 15, 16]}],


In [12]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7633333333333333
{'gamma': 0, 'learning_rate': 0.1, 'max_depth': 14}


In [13]:
from sklearn.externals import joblib

xgb_leave_clf = XGBClassifier(random_state = 42, learning_rate = 0.1, gamma = 3, max_depth = 14, n_jobs = -1)
xgb_leave_clf.fit(X_train.iloc[:, 1:], y_train['leave'])
joblib.dump(xgb_leave_clf, './xgb_leave_clf.pkl')

['./xgb_leave_clf.pkl']

### Ensemble

In [14]:
from sklearn.ensemble import VotingClassifier

ensemble_leave_clf = VotingClassifier(
    estimators = [('rf', rf_leave_clf), ('xgb', xgb_leave_clf)],
    voting = 'soft', n_jobs = -1)

ensemble_leave_clf.fit(X_train.iloc[:, 1:], y_train['leave'])

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features=15,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=750,
                                                     n_jobs=-1, oob_score=False,

In [15]:
from sklearn.externals import joblib

joblib.dump(ensemble_leave_clf, './ensemble_leave_clf.pkl')

['./ensemble_leave_clf.pkl']

## 결제 여부

### RandomForest

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier(random_state=42)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [11, 12, 13, 14]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)

grid_search.fit(X_train.iloc[:, 1:], y_train['amount'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:  1.8min remaining:  2.0min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'m

In [17]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.8580333333333333
{'max_features': 13, 'n_estimators': 300}


In [18]:
from sklearn.externals import joblib

rf_amount_clf = RandomForestClassifier(random_state = 42, max_features = 11, n_estimators = 750, n_jobs = -1)
rf_amount_clf.fit(X_train.iloc[:, 1:], y_train['amount'])
joblib.dump(rf_amount_clf, './rf_amount_clf.pkl')

['./rf_amount_clf.pkl']

### XGBoost 

In [19]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

xgb = XGBClassifier(random_state=42)

param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'max_depth' : [10, 11, 12, 13], 'gamma' : [0, 1, 3]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)

grid_search.fit(X_train.iloc[:, 1:], y_train['amount'])

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  5.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=42, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'gamma': [0, 1, 3],
                          'learning_rate': [0.001, 0.01, 0.1],
                          'max_depth': [10, 11, 12, 13]}],


In [20]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.8575
{'gamma': 0, 'learning_rate': 0.1, 'max_depth': 11}


In [21]:
from sklearn.externals import joblib

xgb_amount_clf = XGBClassifier(random_state = 42, learning_rate = 0.1, gamma = 3, max_depth = 10, n_jobs = -1)
xgb_amount_clf.fit(X_train.iloc[:, 1:], y_train['amount'])
joblib.dump(xgb_amount_clf, './xgb_amount_clf.pkl')

['./xgb_amount_clf.pkl']

### Ensemble

In [22]:
from sklearn.ensemble import VotingClassifier

ensemble_amount_clf = VotingClassifier(
    estimators = [('rf', rf_amount_clf), ('xgb', xgb_amount_clf)],
    voting = 'soft', n_jobs = -1)

ensemble_amount_clf.fit(X_train.iloc[:, 1:], y_train['amount'])

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features=11,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=750,
                                                     n_jobs=-1, oob_score=False,

In [23]:
from sklearn.externals import joblib

joblib.dump(ensemble_amount_clf, './ensemble_amount_clf.pkl')

['./ensemble_amount_clf.pkl']

## 추가 생존 기간

In [24]:
train = pd.merge(X_train, y_train, how = 'left', on = 'acc_id')

In [25]:
train_leave = train[train['leave'] == 1]
train_amount = train[(train['leave'] == 1) & (train['amount'] == 1)]

### RandomForest

In [26]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

rf = RandomForestRegressor(random_state=42)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [16, 17, 18, 19]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_leave.iloc[:, 1:-4], train_leave['survival_time'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:  1.1min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.9min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'max_features': [16, 17, 18, 19],


In [27]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-201.72938098608364
{'max_features': 17, 'n_estimators': 750}


In [28]:
from sklearn.externals import joblib

rf_leave_reg = RandomForestRegressor(random_state = 42, max_features = 18, n_estimators = 750, n_jobs = -1)
rf_leave_reg.fit(train_leave.iloc[:, 1:-4], train_leave['survival_time'])
joblib.dump(rf_leave_reg, './rf_leave_reg.pkl')

['./rf_leave_reg.pkl']

### XGBoost

In [29]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

xgb = XGBRegressor(random_state=42)

param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'max_depth' : [7, 8, 9, 10], 'gamma' : [1, 3, 5]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_leave.iloc[:, 1:-4], train_leave['survival_time'])

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:   53.6s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.5min finished
/home/user3/anaconda3/envs/hun/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/user3/anaconda3/envs/hun/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[14:22:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=42,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'gamma': [1, 3, 5],
                          'learning_rate': [0.001, 0.01, 0.1],
              

In [30]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-208.996915600758
{'gamma': 3, 'learning_rate': 0.1, 'max_depth': 9}


In [31]:
from sklearn.externals import joblib

xgb_leave_reg = XGBRegressor(random_state = 42, learning_rate = 0.1, gamma = 5, max_depth = 8, n_jobs = -1)
xgb_leave_reg.fit(train_leave.iloc[:, 1:-4], train_leave['survival_time'])
joblib.dump(xgb_leave_reg, './xgb_leave_reg.pkl')

[14:22:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


['./xgb_leave_reg.pkl']

### ExtraTrees

In [32]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV

extree = ExtraTreesRegressor(random_state = 42, bootstrap = True)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [19, 20, 21, 22]}
  ]

grid_search = GridSearchCV(extree, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_leave.iloc[:, 1:-4], train_leave['survival_time'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:   36.7s remaining:   42.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   58.8s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=ExtraTreesRegressor(bootstrap=True, criterion='mse',
                                           max_depth=None, max_features='auto',
                                           max_leaf_nodes=None,
                                           min_impurity_decrease=0.0,
                                           min_impurity_split=None,
                                           min_samples_leaf=1,
                                           min_samples_split=2,
                                           min_weight_fraction_leaf=0.0,
                                           n_estimators='warn', n_jobs=None,
                                           oob_score=False, random_state=42,
                                           verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'max_features': [19, 20, 21, 22],
                          'n_estimators': [300, 500, 750]}],
      

In [33]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-203.39616221817192
{'max_features': 21, 'n_estimators': 750}


In [34]:
from sklearn.externals import joblib

extree_leave_reg = ExtraTreesRegressor(random_state = 42, bootstrap = True, max_features = 22, n_estimators = 750, n_jobs = -1)
extree_leave_reg.fit(train_leave.iloc[:, 1:-4], train_leave['survival_time'])
joblib.dump(extree_leave_reg, './extree_leave_reg.pkl')

['./extree_leave_reg.pkl']

### ElasticNet

In [35]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

lr = ElasticNet(random_state = 42)

param_grid = [
    {'alpha' : [0, 0.1, 1, 3, 5], 'l1_ratio' : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}
  ]

grid_search = GridSearchCV(lr, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)
grid_search.fit(train_leave.iloc[:, 1:-4], train_leave['survival_time'])

Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:   11.7s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=1000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=42, selection='cyclic',
                                  tol=0.0001, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'alpha': [0, 0.1, 1, 3, 5],
                          'l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,
                                       0.9]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=2)

In [36]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-238.25250958102515
{'alpha': 0.1, 'l1_ratio': 0.4}


In [37]:
from sklearn.externals import joblib

lr_leave_reg = ElasticNet(random_state = 42, alpha = 0.1, l1_ratio = 0.8)
lr_leave_reg.fit(train_leave.iloc[:, 1:-4], train_leave['survival_time'])
joblib.dump(lr_leave_reg, './lr_leave_reg.pkl')

['./lr_leave_reg.pkl']

## 일 평균 결제 금액

### RandomForest

In [38]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

rf = RandomForestRegressor(random_state=42)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [5, 6, 7, 8]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_amount.iloc[:, 1:-4], train_amount['amount_spent'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:   14.2s remaining:   16.2s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   29.3s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'max_features': [5, 6, 7, 8],
    

In [39]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-1.9007880353495359
{'max_features': 6, 'n_estimators': 500}


In [40]:
from sklearn.externals import joblib

rf_amount_reg = RandomForestRegressor(random_state = 42, max_features = 7, n_estimators = 500, n_jobs = -1)
rf_amount_reg.fit(train_amount.iloc[:, 1:-4], train_amount['amount_spent'])
joblib.dump(rf_amount_reg, './rf_amount_reg.pkl')

['./rf_amount_reg.pkl']

### XGBoost

In [41]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

xgb = XGBRegressor(random_state=42)

param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'max_depth' : [1, 2, 3, 4], 'gamma' : [0, 1, 3, 5]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_amount.iloc[:, 1:-4], train_amount['amount_spent'])

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:   17.4s finished
/home/user3/anaconda3/envs/hun/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/user3/anaconda3/envs/hun/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[14:25:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=42,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'gamma': [0, 1, 3, 5],
                          'learning_rate': [0.001, 0.01, 0.1],
           

In [42]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-1.9764363700601124
{'gamma': 0, 'learning_rate': 0.01, 'max_depth': 2}


In [43]:
from sklearn.externals import joblib

xgb_amount_reg = XGBRegressor(random_state = 42, learning_rate = 0.01, gamma = 3, max_depth = 2, n_jobs = -1)
xgb_amount_reg.fit(train_amount.iloc[:, 1:-4], train_amount['amount_spent'])
joblib.dump(xgb_amount_reg, './xgb_amount_reg.pkl')

[14:25:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


['./xgb_amount_reg.pkl']

### ExtraTrees

In [44]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV

extree = ExtraTreesRegressor(random_state = 42, bootstrap = True)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [6, 7, 8, 9]}
  ]

grid_search = GridSearchCV(extree, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_amount.iloc[:, 1:-4], train_amount['amount_spent'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:    8.1s remaining:    9.2s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   13.9s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=ExtraTreesRegressor(bootstrap=True, criterion='mse',
                                           max_depth=None, max_features='auto',
                                           max_leaf_nodes=None,
                                           min_impurity_decrease=0.0,
                                           min_impurity_split=None,
                                           min_samples_leaf=1,
                                           min_samples_split=2,
                                           min_weight_fraction_leaf=0.0,
                                           n_estimators='warn', n_jobs=None,
                                           oob_score=False, random_state=42,
                                           verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'max_features': [6, 7, 8, 9],
                          'n_estimators': [300, 500, 750]}],
          

In [45]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-1.9511252908936363
{'max_features': 7, 'n_estimators': 750}


In [46]:
from sklearn.externals import joblib

extree_amount_reg = ExtraTreesRegressor(random_state = 42, bootstrap = True, max_features = 7, n_estimators = 300, n_jobs = -1)
extree_amount_reg.fit(train_amount.iloc[:, 1:-4], train_amount['amount_spent'])
joblib.dump(extree_amount_reg, './extree_amount_reg.pkl')

['./extree_amount_reg.pkl']

### ElasticNet

In [47]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

lr = ElasticNet(random_state = 42)

param_grid = [
    {'alpha' : [0, 0.1, 1, 3, 5], 'l1_ratio' : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}
  ]

grid_search = GridSearchCV(lr, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_amount.iloc[:, 1:-4], train_amount['amount_spent'])

Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:    4.0s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=1000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=42, selection='cyclic',
                                  tol=0.0001, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'alpha': [0, 0.1, 1, 3, 5],
                          'l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,
                                       0.9]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=2)

In [48]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-2.0195345870598507
{'alpha': 0.1, 'l1_ratio': 0.3}


In [49]:
from sklearn.externals import joblib

lr_amount_reg = ElasticNet(random_state = 42, alpha = 0.1, l1_ratio = 0.2)
lr_amount_reg.fit(train_amount.iloc[:, 1:-4], train_amount['amount_spent'])
joblib.dump(lr_amount_reg, './lr_amount_reg.pkl')

['./lr_amount_reg.pkl']

# 테스트

In [50]:
path_data_test = os.path.join(os.path.realpath(__file__).rsplit('/', 1)[0], 'preprocess', 'test_preprocess_2.csv')

In [51]:
X_test = pd.read_csv(path_data_test, engine = 'python')

## 이탈 여부

### RandomForest

In [52]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

y_pred_rf = rf_leave_clf.predict(X_test.iloc[:, 1:])
print('acc :', accuracy_score(y_test['leave'], y_pred_rf))
print('recall :', recall_score(y_test['leave'], y_pred_rf))
print('precision :', precision_score(y_test['leave'], y_pred_rf))
print('f1 score :', f1_score(y_test['leave'], y_pred_rf))
confusion_matrix(y_test['leave'], y_pred_rf)

acc : 0.7726
recall : 0.7065096645189958
precision : 0.7694168884587467
f1 score : 0.7366226546212647


array([[4546,  953],
       [1321, 3180]])

### XGBoost

In [53]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

y_pred_xgb = xgb_leave_clf.predict(X_test.iloc[:, 1:])
print('acc :', accuracy_score(y_test['leave'], y_pred_xgb))
print('recall :', recall_score(y_test['leave'], y_pred_xgb))
print('precision :', precision_score(y_test['leave'], y_pred_xgb))
print('f1 score :', f1_score(y_test['leave'], y_pred_xgb))
confusion_matrix(y_test['leave'], y_pred_xgb)

acc : 0.7631
recall : 0.705620973117085
precision : 0.75260663507109
f1 score : 0.7283568398119481


array([[4455, 1044],
       [1325, 3176]])

### Ensemble

In [54]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

y_pred_ensemble = ensemble_leave_clf.predict(X_test.iloc[:, 1:])
print('acc :', accuracy_score(y_test['leave'], y_pred_ensemble))
print('recall :', recall_score(y_test['leave'], y_pred_ensemble))
print('precision :', precision_score(y_test['leave'], y_pred_ensemble))
print('f1 score :', f1_score(y_test['leave'], y_pred_ensemble))
confusion_matrix(y_test['leave'], y_pred_ensemble)

acc : 0.7706
recall : 0.7073983559209065
precision : 0.7652006729151646
f1 score : 0.7351650888940199


array([[4522,  977],
       [1317, 3184]])

## 결제 여부

### RandomForest

In [55]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

y_pred_rf = rf_amount_clf.predict(X_test.iloc[:, 1:])
print('acc :', accuracy_score(y_test['amount'], y_pred_rf))
print('recall :', recall_score(y_test['amount'], y_pred_rf))
print('precision :', precision_score(y_test['amount'], y_pred_rf))
print('f1 score :', f1_score(y_test['amount'], y_pred_rf))
confusion_matrix(y_test['amount'], y_pred_rf)

acc : 0.8574
recall : 0.921885753613214
precision : 0.8464454976303317
f1 score : 0.8825564157469937


array([[3216,  972],
       [ 454, 5358]])

### XGBoost

In [56]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

y_pred_xgb = xgb_amount_clf.predict(X_test.iloc[:, 1:])
print('acc :', accuracy_score(y_test['amount'], y_pred_xgb))
print('recall :', recall_score(y_test['amount'], y_pred_xgb))
print('precision :', precision_score(y_test['amount'], y_pred_xgb))
print('f1 score :', f1_score(y_test['amount'], y_pred_xgb))
confusion_matrix(y_test['amount'], y_pred_xgb)

acc : 0.8561
recall : 0.9091534755677908
precision : 0.8529459241323648
f1 score : 0.8801532439410343


array([[3277,  911],
       [ 528, 5284]])

### Ensemble

In [57]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

y_pred_ensemble = ensemble_amount_clf.predict(X_test.iloc[:, 1:])
print('acc :', accuracy_score(y_test['amount'], y_pred_ensemble))
print('recall :', recall_score(y_test['amount'], y_pred_ensemble))
print('precision :', precision_score(y_test['amount'], y_pred_ensemble))
print('f1 score :', f1_score(y_test['amount'], y_pred_ensemble))
confusion_matrix(y_test['amount'], y_pred_ensemble)

acc : 0.8572
recall : 0.9172401927047488
precision : 0.849155782096209
f1 score : 0.8818858560794044


array([[3241,  947],
       [ 481, 5331]])

## 추가 생존 기간

### RandomForest

In [58]:
from sklearn.metrics import mean_squared_error

y_pred_rf = rf_leave_reg.predict(X_test.iloc[:, 1:])
print('mse :', mean_squared_error(y_test['survival_time'], y_pred_rf))

mse : 823.7027374009992


### XGBoost

In [59]:
from sklearn.metrics import mean_squared_error

y_pred_xgb = xgb_leave_reg.predict(X_test.iloc[:, 1:])
print('mse :', mean_squared_error(y_test['survival_time'], y_pred_xgb))

mse : 830.4630963338064


### ExtraTrees

In [60]:
from sklearn.metrics import mean_squared_error

y_pred_extree = extree_leave_reg.predict(X_test.iloc[:, 1:])
print('mse :', mean_squared_error(y_test['survival_time'], y_pred_extree))

mse : 840.4614706520662


### ElasticNet

In [61]:
from sklearn.metrics import mean_squared_error

y_pred_lr = lr_leave_reg.predict(X_test.iloc[:, 1:])
print('mse :', mean_squared_error(y_test['survival_time'], y_pred_lr))

mse : 835.941754613301


## 일 평균 결제 금액

### RandomForest

In [62]:
from sklearn.metrics import mean_squared_error

y_pred_rf = rf_amount_reg.predict(X_test.iloc[:, 1:])
print('mse :', mean_squared_error(y_test['amount_spent'], y_pred_rf))

mse : 0.8968534380831418


### XGBoost

In [63]:
from sklearn.metrics import mean_squared_error

y_pred_xgb = xgb_amount_reg.predict(X_test.iloc[:, 1:])
print('mse :', mean_squared_error(y_test['amount_spent'], y_pred_xgb))

mse : 0.7044065196327494


### ExtraTrees

In [64]:
from sklearn.metrics import mean_squared_error

y_pred_extree = extree_amount_reg.predict(X_test.iloc[:, 1:])
print('mse :', mean_squared_error(y_test['amount_spent'], y_pred_extree))

mse : 0.8328581870360369


### ElasticNet

In [65]:
from sklearn.metrics import mean_squared_error

y_pred_lr = lr_amount_reg.predict(X_test.iloc[:, 1:])
print('mse :', mean_squared_error(y_test['amount_spent'], y_pred_lr))

mse : 0.7004694475856131


# 베이지안 최적화

In [66]:
from sklearn.externals import joblib

ensemble_leave_clf = joblib.load('./ensemble_leave_clf.pkl')

ensemble_amount_clf = joblib.load('./ensemble_amount_clf.pkl')

rf_leave_reg = joblib.load('./rf_leave_reg.pkl')
xgb_leave_reg = joblib.load('./xgb_leave_reg.pkl')
extree_leave_reg = joblib.load('./extree_leave_reg.pkl')
lr_leave_reg = joblib.load('./lr_leave_reg.pkl')

rf_amount_reg = joblib.load('./rf_amount_reg.pkl')
xgb_amount_reg = joblib.load('./xgb_amount_reg.pkl')
extree_amount_reg = joblib.load('./extree_amount_reg.pkl')
lr_amount_reg = joblib.load('./lr_amount_reg.pkl')

[14:25:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:25:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [67]:
y_test.iloc[:, :3].to_csv('./true.csv', index = False)

In [68]:
def predict(model1, model2, model3, model4, model5, model6, model7, model8, model9, model10, data,
            w1, w2, w3, w4, w5, w6, w7, w8):
    '''
    model1 : 생존 여부 예측(classifier)
    model2 : 결제 여부 예측(classifier)
    model3~6 : 추가 생존 기간 예측(regressor)
    model7~10 : 일 평균 결제 금액 예측(regressor)
    data : test data
    w1~w4 : 추가 생존 기간 예측 모델 weight
    w5~w8 : 일 평균 결제 금액 예측 모델 weight
    '''
    
    pred1 = model1.predict(data.iloc[:, 1:])
    pred2 = model2.predict(data.iloc[:, 1:])
    pred3 = ((w1 * model3.predict(data.iloc[:, 1:])) + (w2 * model4.predict(data.iloc[:, 1:]))
            + (w3 * model5.predict(data.iloc[:, 1:])) + (w4 * model6.predict(data.iloc[:, 1:])))
    pred4 = ((w5 * model7.predict(data.iloc[:, 1:])) + (w6 * model8.predict(data.iloc[:, 1:]))
            + (w7 * model9.predict(data.iloc[:, 1:])) + (w8 * model10.predict(data.iloc[:, 1:])))
    
    pred3[pred1 == 0] = 64
    pred4[pred2 == 0] = 0
    pred3[pred3 <= 1] = 1
    pred3[pred3 >= 64] = 64
    pred4[pred4 <= 0] = 0
    
    array = np.concatenate([data.iloc[:, 0].values.reshape(-1, 1), pred3.reshape(-1, 1), pred4.reshape(-1, 1)], axis = 1)
    df = pd.DataFrame(array)
    df.columns = ['acc_id', 'survival_time', 'amount_spent']
    df['acc_id'] = df['acc_id'].astype('int32')
    df['survival_time'] = round(df['survival_time']).astype('int32')
    
    return df

In [69]:
from score_function import score_function

def black_box_function(w1, w2, w3, w4, w5, w6, w7, w8):
    test_pred = predict(model1 = ensemble_leave_clf, model2 = ensemble_amount_clf, 
                            model3 = rf_leave_reg, model4 = xgb_leave_reg,
                            model5 = extree_leave_reg, model6 = lr_leave_reg,
                            model7 = rf_amount_reg, model8 = xgb_amount_reg,
                            model9 = extree_amount_reg, model10 = lr_amount_reg,
                            data = X_test,
                            w1 = w1, w2 = w2, w3 = w3, w4 = w4, w5 = w5, w6 = w6, w7 = w7, w8 = w8)
    
    test_pred.to_csv('./y_pred.csv', index = False)
    test_score = score_function('./y_pred.csv', './true.csv')
    
    return test_score

In [70]:
from bayes_opt import BayesianOptimization

pbounds = {'w1' : (0, 1), 'w2' : (0, 1), 'w3' : (0, 1), 'w4' : (0, 1),
          'w5' : (0.05, 1.5), 'w6' : (0.05, 1.5), 'w7' : (0.05, 1.5), 'w8' : (0.05, 1.5)}

optimizer = BayesianOptimization(
    f = black_box_function,
    pbounds = pbounds,
    random_state = 42,
)

In [71]:
optimizer.maximize(
    init_points = 50,
    n_iter = 500
)

|   iter    |  target   |    w1     |    w2     |    w3     |    w4     |    w5     |    w6     |    w7     |    w8     |
-------------------------------------------------------------------------------------------------------------------------
1681.2155036820757
|  1        |  1.681e+0 |  0.3745   |  0.9507   |  0.732    |  0.5987   |  0.2762   |  0.2762   |  0.1342   |  1.306    |
2017.3409292846748
|  2        |  2.017e+0 |  0.6011   |  0.7081   |  0.02058  |  0.9699   |  1.257    |  0.3579   |  0.3136   |  0.3159   |
3965.5452909079777
|  3        |  3.966e+0 |  0.3042   |  0.5248   |  0.4319   |  0.2912   |  0.9372   |  0.2523   |  0.4736   |  0.5812   |
3020.3969795688736
|  4        |  3.02e+03 |  0.4561   |  0.7852   |  0.1997   |  0.5142   |  0.909    |  0.1174   |  0.9309   |  0.2973   |
1655.0325787536954
|  5        |  1.655e+0 |  0.06505  |  0.9489   |  0.9656   |  0.8084   |  0.4917   |  0.1916   |  1.042    |  0.6882   |
3774.112850243528
|  6        |  3.774e+0 |  0.122 

5487.981055590978
|  37       |  5.488e+0 |  0.2865   |  0.5908   |  0.0305   |  0.03735  |  1.243    |  0.5723   |  0.2342   |  0.8073   |
3619.2252774975127
|  38       |  3.619e+0 |  0.77     |  0.2158   |  0.6229   |  0.08535  |  0.1249   |  0.8205   |  0.8339   |  0.9743   |
2027.9800896917404
|  39       |  2.028e+0 |  0.7261   |  0.9759   |  0.5163   |  0.323    |  1.203    |  0.4427   |  0.6865   |  0.1638   |
1764.6129353090694
|  40       |  1.765e+0 |  0.02535  |  0.9626   |  0.836    |  0.696    |  0.643    |  0.3013   |  0.2768   |  0.4129   |
2482.869637863869
|  41       |  2.483e+0 |  0.5492   |  0.7146   |  0.6602   |  0.2799   |  1.435    |  1.12     |  0.8538   |  0.937    |
1850.4420680167807
|  42       |  1.85e+03 |  0.4196   |  0.2477   |  0.356    |  0.7578   |  0.07087  |  0.2183   |  0.1167   |  0.1091   |
2619.6621589201523
|  43       |  2.62e+03 |  0.8555   |  0.7037   |  0.4742   |  0.09783  |  0.7628   |  0.7365   |  0.3011   |  0.6791   |
3722.0407032912

5910.3626508223415
|  74       |  5.91e+03 |  0.0347   |  0.2602   |  0.1237   |  0.1116   |  0.5443   |  0.6747   |  1.308    |  0.7736   |
5924.537392843931
|  75       |  5.925e+0 |  0.03645  |  0.3067   |  0.09293  |  0.1252   |  0.5837   |  0.6502   |  1.258    |  0.8015   |
5924.729933797689
|  76       |  5.925e+0 |  0.04483  |  0.2828   |  0.1378   |  0.1256   |  0.5837   |  0.6413   |  1.273    |  0.7834   |
5937.686650841916
|  77       |  5.938e+0 |  0.03605  |  0.3034   |  0.1105   |  0.08126  |  0.6253   |  0.6913   |  1.276    |  0.7769   |
5920.734248582808
|  78       |  5.921e+0 |  0.03504  |  0.295    |  0.1165   |  0.07741  |  0.6052   |  0.6698   |  1.268    |  0.7849   |
5997.073284966401
|  79       |  5.997e+0 |  0.03195  |  0.3119   |  0.08362  |  0.1489   |  0.6542   |  0.6686   |  1.357    |  0.7785   |
5995.593434413673
|  80       |  5.996e+0 |  0.03232  |  0.3108   |  0.08426  |  0.1487   |  0.6544   |  0.6694   |  1.354    |  0.7785   |
5988.38194636937
| 

6221.894696294243
|  111      |  6.222e+0 |  0.0      |  0.517    |  0.0      |  0.04654  |  0.7054   |  0.8024   |  1.5      |  1.125    |
6220.498132739546
|  112      |  6.22e+03 |  0.0      |  0.5176   |  0.0      |  0.04544  |  0.7045   |  0.8045   |  1.5      |  1.123    |
6218.525731158271
|  113      |  6.219e+0 |  0.0      |  0.4996   |  0.0      |  0.05414  |  0.7174   |  0.7769   |  1.5      |  1.144    |
6224.35840753046
|  114      |  6.224e+0 |  0.0      |  0.5166   |  0.0      |  0.04857  |  0.71     |  0.7996   |  1.5      |  1.132    |
6242.428630253643
|  115      |  6.242e+0 |  0.0      |  0.5317   |  0.0      |  0.05133  |  0.7399   |  0.8149   |  1.5      |  1.18     |
6244.946082204776
|  116      |  6.245e+0 |  0.0      |  0.5331   |  0.0      |  0.05135  |  0.7448   |  0.817    |  1.5      |  1.19     |
6256.550435506308
|  117      |  6.257e+0 |  0.0      |  0.5313   |  0.0      |  0.05162  |  0.7541   |  0.817    |  1.5      |  1.226    |
6269.464640929383
|  

6397.618244893149
|  148      |  6.398e+0 |  0.0      |  0.5684   |  0.0      |  0.0      |  1.123    |  1.016    |  1.5      |  1.5      |
6417.787603879385
|  149      |  6.418e+0 |  0.0      |  0.538    |  0.0      |  0.0      |  1.237    |  1.104    |  1.5      |  1.5      |
6409.619330088153
|  150      |  6.41e+03 |  0.0      |  0.5473   |  0.0      |  0.0      |  1.203    |  1.078    |  1.5      |  1.5      |
6419.710287556044
|  151      |  6.42e+03 |  0.0      |  0.5177   |  0.0      |  0.0      |  1.298    |  1.155    |  1.5      |  1.5      |
6423.613172072432
|  152      |  6.424e+0 |  0.0      |  0.5288   |  0.0      |  0.0      |  1.275    |  1.133    |  1.5      |  1.5      |
6424.259732759553
|  153      |  6.424e+0 |  0.0      |  0.5349   |  0.0      |  0.0      |  1.278    |  1.132    |  1.5      |  1.5      |
6421.441317390576
|  154      |  6.421e+0 |  0.0      |  0.5398   |  0.0      |  0.0      |  1.281    |  1.132    |  1.5      |  1.5      |
6414.431122536468
| 

...

6388.7626808463365
|  375      |  6.389e+0 |  0.0      |  0.4562   |  0.0      |  0.09963  |  1.166    |  1.5      |  1.394    |  1.436    |
6392.257895659341
|  376      |  6.392e+0 |  0.04948  |  0.5716   |  0.0      |  0.0      |  1.212    |  1.23     |  1.5      |  1.5      |
6419.972348429497
|  377      |  6.42e+03 |  0.0      |  0.5817   |  0.0      |  0.0      |  1.34     |  1.425    |  1.5      |  1.5      |
6034.484200250484
|  378      |  6.034e+0 |  0.06528  |  0.2175   |  0.04074  |  0.1863   |  0.7182   |  0.5797   |  1.43     |  0.8757   |
6033.205430614142
|  379      |  6.033e+0 |  0.05869  |  0.637    |  0.1783   |  0.008214 |  1.18     |  1.397    |  1.449    |  1.308    |
5991.0598058654805
|  380      |  5.991e+0 |  0.03007  |  0.319    |  0.05429  |  0.1489   |  0.6477   |  0.6591   |  1.385    |  0.7974   |
6422.7197765792125
|  381      |  6.423e+0 |  0.0      |  0.5703   |  0.0      |  0.001213 |  1.323    |  1.414    |  1.5      |  1.5      |
2854.789069641665

6422.056118941074
|  413      |  6.422e+0 |  0.0      |  0.5744   |  0.0      |  0.0      |  1.321    |  1.405    |  1.5      |  1.5      |
5906.104040133306
|  414      |  5.906e+0 |  0.1888   |  0.3865   |  0.1864   |  0.1584   |  1.056    |  1.299    |  1.345    |  1.453    |
6359.675012442566
|  415      |  6.36e+03 |  0.0      |  0.468    |  0.05425  |  0.1197   |  1.074    |  1.5      |  1.5      |  1.463    |
3326.8408828771626
|  416      |  3.327e+0 |  0.4249   |  0.05337  |  0.3543   |  0.8846   |  0.6297   |  0.7303   |  0.5159   |  0.4966   |
6346.842804802672
|  417      |  6.347e+0 |  0.0      |  0.434    |  0.02537  |  0.1167   |  1.084    |  1.205    |  1.272    |  1.427    |
6158.121563698851
|  418      |  6.158e+0 |  0.07564  |  0.6444   |  0.01345  |  0.07001  |  0.9134   |  1.117    |  1.42     |  1.364    |
2302.7948546459806
|  419      |  2.303e+0 |  0.5983   |  0.9479   |  0.03384  |  0.6646   |  1.412    |  0.3889   |  1.438    |  1.336    |
6289.256170278954


6335.212100861771
|  451      |  6.335e+0 |  0.04905  |  0.4575   |  0.02695  |  0.04825  |  0.9838   |  1.165    |  1.334    |  1.421    |
6217.61559660244
|  452      |  6.218e+0 |  0.02086  |  0.2688   |  0.002204 |  0.3151   |  1.023    |  0.8228   |  1.341    |  1.08     |
1951.5640494811917
|  453      |  1.952e+0 |  0.3255   |  0.6124   |  0.8345   |  0.6411   |  0.1297   |  0.4178   |  0.7291   |  1.3      |
6388.097548529246
|  454      |  6.388e+0 |  0.0      |  0.3207   |  0.0      |  0.1936   |  1.278    |  1.5      |  1.418    |  1.436    |
6203.476670691796
|  455      |  6.203e+0 |  0.1282   |  0.3608   |  0.04466  |  0.04588  |  0.7116   |  1.078    |  1.305    |  1.194    |
6253.137305659121
|  456      |  6.253e+0 |  0.03974  |  0.245    |  0.04471  |  0.1807   |  1.144    |  0.7813   |  1.1      |  1.367    |
6092.6967107635
|  457      |  6.093e+0 |  0.03414  |  0.2132   |  0.2493   |  0.2613   |  0.9134   |  0.7443   |  1.388    |  1.072    |
5911.366521463866
|  4

6285.223814455634
|  489      |  6.285e+0 |  0.02814  |  0.3037   |  0.1903   |  0.09117  |  0.9359   |  0.964    |  1.492    |  1.066    |
6396.02581598958
|  490      |  6.396e+0 |  0.000296 |  0.3244   |  0.0      |  0.2096   |  1.225    |  1.499    |  1.473    |  1.413    |
6031.9180938075615
|  491      |  6.032e+0 |  0.05377  |  0.224    |  0.3499   |  0.1613   |  1.005    |  0.9905   |  1.259    |  0.866    |
6294.406246013111
|  492      |  6.294e+0 |  0.07003  |  0.1029   |  0.333    |  0.07009  |  0.9334   |  0.8921   |  1.467    |  1.26     |
6416.071256949695
|  493      |  6.416e+0 |  0.0      |  0.4676   |  0.0      |  0.0863   |  1.269    |  1.5      |  1.5      |  1.5      |
4464.060931681349
|  494      |  4.464e+0 |  0.2705   |  0.04623  |  0.2638   |  0.6699   |  0.2439   |  1.092    |  0.1527   |  0.9227   |
6279.076530737204
|  495      |  6.279e+0 |  0.02637  |  0.3159   |  0.1522   |  0.1384   |  0.9196   |  1.016    |  1.321    |  1.335    |
6407.06036712785
|  

6381.234010902029
|  527      |  6.381e+0 |  0.000118 |  0.3255   |  0.001041 |  0.1947   |  1.144    |  1.499    |  1.498    |  1.423    |
1708.2048845616675
|  528      |  1.708e+0 |  0.8909   |  0.04281  |  0.9083   |  0.7748   |  0.3755   |  0.251    |  0.8854   |  1.311    |
6414.881965476896
|  529      |  6.415e+0 |  0.0      |  0.4112   |  0.0      |  0.1263   |  1.306    |  1.5      |  1.5      |  1.499    |
6379.402899437648
|  530      |  6.379e+0 |  0.0      |  0.2928   |  0.09679  |  0.1458   |  1.152    |  0.979    |  1.5      |  1.314    |
6222.974051830132
|  531      |  6.223e+0 |  0.1811   |  0.5049   |  0.04456  |  0.036    |  1.011    |  1.07     |  1.492    |  1.428    |
6215.603105016439
|  532      |  6.216e+0 |  0.01677  |  0.1426   |  0.06504  |  0.2614   |  0.9708   |  0.7009   |  1.274    |  1.344    |
5786.71330389022
|  533      |  5.787e+0 |  0.082    |  0.5474   |  0.1449   |  0.1732   |  0.5755   |  0.9903   |  1.211    |  1.412    |
6376.9735646866875
|

In [72]:
print(optimizer.max)

{'target': 6437.736119126102, 'params': {'w1': 0.0, 'w2': 0.5361283013725007, 'w3': 0.0, 'w4': 0.0, 'w5': 1.3675901396673014, 'w6': 1.1385865634353487, 'w7': 1.5, 'w8': 1.4999999990570498}}


베이지안 최적화를 통한 weight값이 실제 예측에 쓰인 값과 다를 수 있는데 이는 매 실행마다 결과가 다르기 때문이며 성능에는 크게 영향을 미치지 않는 것으로 판단됩니다. 이를 참고해 주시기 바랍니다.